In [2]:
import bk.load
import bk.compute

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import neuroseries as nts

from scipy.stats import zscore
from tqdm import tqdm

import umap
import umap.plot

In [3]:
bk.load.current_session_linux()
neurons,metadata = bk.load.spikes()
states = bk.load.states()
states.update({'sleep':states['Rem'].union(states['sws'])})

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [94]:
actmat = bk.compute.binSpikes(neurons[metadata.Region == 'BLA'],0.50,as_Tsd=True)
actmat = actmat.restrict(states['sleep'])

In [106]:
plt.imshow(zscore(actmat.values[:5000,:],1),aspect='auto',interpolation='none')

In [95]:
in_rem = states['Rem'].in_interval(actmat)
in_sws = states['sws'].in_interval(actmat)

state_vector = np.empty(len(actmat),dtype='object')
state_vector[in_rem > 0] = 'REM'
state_vector[in_sws > 0] = 'NREM'
state_vector[state_vector == None] = 'None'

In [121]:
%matplotlib qt
mapper = umap.UMAP().fit(zscore(actmat.values,0))
umap.plot.points(mapper, labels=state_vector)

<AxesSubplot:>

In [120]:
np.nanmean(zscore(actmat.values,0),1)

array([0.10732339, 0.01531809, 0.33556376, ..., 0.1775795 , 0.13485741,
       0.07884294])

In [78]:
def dimreduction(neurons,binSize = 0.025,baseline_intervals = None):
    from scipy.stats import zscore

    t,actmat = bk.compute.binSpikes(neurons,binSize=binSize)
    actmat = zscore(actmat,1)
    actmat = actmat.T
    actmat = nts.TsdFrame(t,actmat,time_units='s')

    if baseline_intervals is not None:
        c = np.corrcoef(actmat.restrict(baseline_intervals).values.T)
    else: 
        c = np.corrcoef(actmat.values.T)
    eigvalues,vectors = np.linalg.eig(c)
    
    #The documentation of numpy precise that eigvalues are not sorted.
    order = np.argsort(eigvalues)[::-1]
    eigvalues = eigvalues[order]
    vectors = vectors[:,order]
    
    projections = np.zeros_like(actmat.values.T)
    for i,v in enumerate(vectors.T):
        projections[i] = actmat.values.dot(v)
    
    return t,projections